# Make final classifier using mutant concentration
**Goal**: Identify patients that don't respond.
These patients we don't have to treat.

Perform the following steps:

1) Combine data with CNV.

2) Compare harmonic versus delta.

3) Compare clinical data versus clinical and genomic data.

4) Select best classification model based on AUC.

5) For this best model, compare number of molecules.

7) Make plots for parameter importance.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV

from pipelines import benchmark_pipelines, build_classifier_pipelines, pipeline_Freeman, get_classifier_init_params
from transform import generate_data_pairs, generate_model_data_pairs
from views import compare_prognostic_value_genomic_information, view_linear_model_freeman

# 1) Combine data

In [2]:
from source import read_preprocessed_data
from transform import combine_tsv_files

In [3]:
# Difference genomic variable.
X_diff, y = combine_tsv_files(
    "output/all__gene__difference__No. Mutant Molecules per mL.tsv",
    "output/all__gene__difference__CNV Score.tsv",
)

# 2) Select best model

In [4]:
y_resp = y["response_grouped"]

response_labels = ['non responder (sd+pd)', 'responder (pr+cr)', 'non evaluable (ne)']
pos_label = 'responder (pr+cr)'
y_resp = y_resp == pos_label

In [5]:
ps_class = build_classifier_pipelines()

In [ ]:
# Calculate performance using nested loop cross validation.
b_difference = benchmark_pipelines(
    ps_class, X_diff, y_resp, metric='roc_auc', verbose=False,
)
b_difference.to_csv("difference_molecules_benchmask.csv")

--------------------
DummyClassifier(random_state=1234, strategy='most_frequent') :
params: {}

--------------------
DummyClassifier(random_state=1234, strategy='most_frequent') :
params: {'statistical_filter__alpha': [0.05, 0.1, 0.2, 0.4]}

Removing 68/69 numeric columns.
Removing 67/69 numeric columns.
Removing 67/69 numeric columns.
Removing 68/69 numeric columns.
Removing 65/69 numeric columns.
--------------------
DummyClassifier(random_state=1234, strategy='most_frequent') :
params: {}

--------------------
LogisticRegression(class_weight='balanced', max_iter=5000, random_state=1234,
                   solver='newton-cg') :
params: {'estimator__C': [0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.175, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 4.0]}

--------------------
LogisticRegression(class_weight='balanced', max_iter=5000, random_state=1234,
                   solver='newton-cg') :
params: {'statistical_filter__alpha': [0.05, 0.1, 0.2, 0.4], 'estimator__C': [0.005, 0.01, 0.025, 0.05, 0.075, 0

In [ ]:
b_difference = pd.read_csv('difference_molecules_benchmask.csv')
b_difference.set_index('Unnamed: 0', inplace=True)
b_difference

The prediction with difference instead of harmonic mean is therefore marginally worse.

In [ ]:
b = b_difference
plt.rc('font', family='serif')
import matplotlib
context = {
#     'font.size': 22,
    'lines.linewidth': 4,
    'figure.autolayout': True,
    'xtick.labelsize': 'large',
    'ytick.labelsize': 'large',
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'xx-large',
    'axes.titlesize': 'xx-large',
}
# fig = plt.figure(figsize=(4, 3))
with plt.rc_context(context):
    fig = plt.figure(figsize=(6, 6))
    degrees=75
    plt.ylabel('AUC ROC')
    plt.xticks(rotation=degrees)
    plt.ylim([0.0, 1.0])
    plt.gca().fill_between([-1, 8], 0, 0.5, facecolor='grey', edgecolor='k', alpha=0.25, hatch='\\')
    plt.errorbar(
        b.index, 
        b['Richard mean'], 
        yerr=b['Richard std'], 
        fmt='s-', 
        markersize=10,
        label='clinical',
    )
    plt.errorbar(
        b.index, 
        b['Freeman mean'], 
        yerr=b['Freeman std'], 
        fmt='o--', 
        markersize=10,
        label='clinical + genomic',
    )
    plt.legend(frameon=False)
    plt.tight_layout()
plt.savefig('figs/comparison_classifiers.png', bbox_inches = 'tight')
plt.savefig('figs/comparison_classifiers.eps', bbox_inches = 'tight')

# 4) Choose model logistic regression.
The logistic regression model doesn't have the best accuracy in absolute terms, but is much simpler than the RandomForest. So by occams razor, we pick it.

In [ ]:
random_state=1234

Estimator = LogisticRegression
kwargs = get_classifier_init_params(Estimator, random_state=random_state)
model = pipeline_Freeman(Estimator(**kwargs))

# 7) Make plots of the model

In [ ]:
figure_filenames = ("logistic_regression_clinical_freeman", "logistic_regression_genetic_freeman")
view_linear_model_freeman(
    X_diff, 
    y_resp, 
    model, 
    filenames=figure_filenames, 
)

# Amount of genetic information.

In [ ]:
random_state=1234

Estimator = LogisticRegression
kwargs = get_classifier_init_params(Estimator, random_state=random_state)
classifier = Estimator(**kwargs)

In [ ]:
mutant_data_pairs = generate_data_pairs(
    filename_prefix="output/all__gene", snv_type="No. Mutant Molecules per mL"
)
vaf_data_pairs = generate_data_pairs(
    filename_prefix="output/all__gene", snv_type="Allele Fraction"
)
model_mutant_data_pairs = generate_model_data_pairs(mutant_data_pairs, model=classifier)
model_vaf_data_pairs = generate_model_data_pairs(vaf_data_pairs, model=classifier)
compare_prognostic_value_genomic_information(model_mutant_data_pairs, plot_label="Mutant concentration", fmt='-')
compare_prognostic_value_genomic_information(model_vaf_data_pairs, plot_label='Allele fraction', fmt='--')
plt.savefig('figs/comparison_genomic_data.png', bbox_inches="tight")